# Classification Power Encoding

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
#%matplotlib notebook
#%matplotlib inline
#%load_ext autoreload
#%autoreload 2
from brainpipe.classification import *
from brainpipe.system import study
from brainpipe.feature import power, amplitude, sigfilt

from brainpipe.statistics import *
from os import path
import mne
from mne.viz import plot_topomap
from mne import pick_types, find_layout
import random as rand

## Creating Rest cond 500ms

In [ ]:
# Rest = 500ms out of 1s 250-750ms
st = study('Olfacto')
subjects = ['CHAF', 'SEMC', 'VACJ','PIRJ', 'MICP', 'LEFC']

for su in subjects:
    file = su+'_E1E2_concat_allfilter1_bipo.npz'
    loadname = path.join(st.path, 'database/TS_E_all_cond_by_block_trigs_th40_art400_30_250', file)
    print('-> Select data in: '+loadname)
    mat = np.load(loadname)['x']

    #Generate the files
    print (mat.shape)
    rest = mat[:,128:384,:]
    print (rest.shape,)

    #save the files
    saverest = loadname.replace('_E1', '_rest_E1').replace('250', '250/Classif_new')
    np.savez(saverest, x=rest)

## Creating expectation and odor perception conditions

In [ ]:
#Odor = expectation + perception
#Expect (0.5s before) Percept (3s following)

st = study('Olfacto')

files = st.search('odor_E1E2_concat_allfilter1_bipo.npz', folder='database/TS_E_all_cond_by_block_trigs_th40_art400_30_250')

for fi in files:
    loadname = path.join(st.path, 'database/TS_E_all_cond_by_block_trigs_th40_art400_30_250', fi)
    print('-> Select data in: '+loadname)
    mat = np.load(loadname)['x']

    #Generate the files
    print (mat.shape)
    expect = mat[:,:256,:]
    percept = mat[:,256:,:]
    print (expect.shape, percept.shape)

    #save the files
    saveexpect = loadname.replace('odor', 'expect').replace('250', '250/Classif_new')
    savepercept = loadname.replace('odor', 'percept').replace('250', '250/Classif_new')
    np.savez(saveexpect, x=expect)
    np.savez(savepercept, x=percept)

## Creating percept by second

In [ ]:
# Percept (3s following)
# creation of 24 windows of 500ms with 25% d'overlap (décale de 125ms)

st = study('Olfacto')
files = st.search('_percept_E1E2_concat_allfilter1_bipo.npz', folder='database/TS_E_all_cond_by_block_trigs_th40_art400_30_250/Classif_new/')

for fi in files:
    loadname = path.join(st.path, 'database/TS_E_all_cond_by_block_trigs_th40_art400_30_250/Classif_new/', fi)
    mat = np.load(loadname)['x']
    n_windows = int((mat.shape[1]-64*3) / 64) #64 samples = 125ms
    print ('Size of Percept file: ', mat.shape, 'Length of data :', mat.shape[1], 'Nb of windows: ', n_windows)

    for i in range(n_windows):
        percept = mat[:,(i)*64:256+(i)*64,:]
        print (i, 'bornes prises :', (i)*64, 256+(i)*64)
        print ('file size : ', percept.shape)
        
        filename = loadname.replace('percept', 'percept_'+str(i))
        np.savez(filename, x=percept)

## Generate mean power files

In [ ]:
# Get all files include in the database
st = study('Olfacto')
files = st.search('_E1E2_concat_allfilter1_bipo_power.npz', folder='feature/Power_Encoding_Rest_Expect_Percept_500ms_windows')
n_freq = 6

for fi in files:
    for freq in range(0, n_freq, 1):
        # Load file (nb_freq, n_elec, n_pts, n_trials):
        loadname = path.join(st.path, 'feature/Power_Encoding_Rest_Expect_Percept_500ms_windows', fi)
        print('-> Compute mean on: '+loadname)
        mat = np.load(loadname)
        x, fname = mat['xpow'], mat['fname']
        x = x[freq,:,:,:] #selection of the freq range
        print (x.shape)
        x = np.mean(x, axis = 1) #mean across time points
        print (x.shape)
        x = x.swapaxes(0, 1)
        print (x.shape) #trials and elecs
        savename = loadname.replace('_power.npz', '_meanpow_'+str(fname[freq])+'.npz')
        #print (savename)
        np.savez(savename, x=x)

## Import & concatenate power files

In [ ]:
# Importing files 
st = study('Olfacto')
pathfiles = path.join(st.path, 'feature/MeanPower_E_all_cond_by_block_trigs_filter1_500art/')
elecfiles = path.join(st.path, 'database/TS_E_all_cond_by_block_trigs_filter1_500art/Odor_rest/')
path2save = path.join(st.path, 'classified/TS_E_all_cond_by_block_trigs_filter1_500art')

#theta alpha gamma30-60 gamma60-120
odor_data = np.load(path.join(pathfiles, 'SEMC_expect_E1E2_concat_allfilter1_bipo_meanpow_delta.npz'))['x']
rest_data = np.load(path.join(pathfiles, 'SEMC_rest_E1E2_concat_allfilter1_bipo_meanpow_delta.npz'))['x']
print (odor_data.shape, rest_data.shape)
elec = np.load(path.join(elecfiles, 'SEMC_odor_E1E2_concat_allfilter1_bipo.npz'))['channel']
print (elec.shape)
#create a data matrix, concatenate along the trial dimension
odor_rest = np.concatenate((odor_data, rest_data), axis=0)
print (odor_rest.shape)

#create label vector (0 for rest and 1 for odor)
label = [1]*odor_data.shape[0] + [0]*rest_data.shape[0]


## Classifier and Cross validation

In [ ]:
# Define a cross validation:
cv = defCv(label, cvtype='skfold', n_folds=3, rep=50)
print (cv)

# Define classifier technique
clf = defClf(label, clf='svm') #,n_tree=200, random_state=100)
print (clf)

#Classify rest and odor
cl = classify(label, clf=clf, cvtype=cv)
#print(cl)

# Evaluate the classifier on data:
da,pvalue,daperm = cl.fit(odor_rest, grp=elec, n_perm=100,method='label_rnd')
da_mean = np.mean(da,axis=0)

In [ ]:
da_mean.shape

In [ ]:
fig1 = plt.figure(1, figsize=(20,10))

cl.daplot(da_mean, daperm=daperm, chance_method='perm', rmax=['top', 'right'],
        dpax=['bottom', 'left'], cmap='magma', ylim=[40,100], chance_unique=True,chance_level = 0.01,
        chance_color='darkgreen')

## Calculate classifier for all freq bands and subjects + Figures
### Analysis subject by subject

In [15]:
# Importing files 
st = study('Olfacto')
pathfiles = path.join(st.path, 'feature/2_Power_Encoding_Odor_rest_th40_art400_30_250_across_time_nooverlap/')
elecfiles = path.join(st.path, 'database/TS_E_all_cond_by_block_trigs_th40_art400_30_250_5s_concatOK/')
path2save = path.join(st.path, 'classified/1_Classif_Windows_Encoding_th40_art400_30_250_by_label_500ms_windows/')

subjects = ['CHAF','SEMC','VACJ','PIRJ', 'MICP', 'LEFC']
freq_bands = ['delta', 'theta', 'alpha', 'beta', 'gamma30-60', 'gamma60-150']
nfreq, nwin = 6, 7
classifier = 'lda'

for su in subjects:
    odor_file = np.load(pathfiles+su+'_E1E2_concat_all_bipo_new_odor_power.npz')['xpow']
    print (su, odor_file.shape)
    rest_file = np.load(pathfiles+su+'_E1E2_concat_all_bipo_new_rest_power.npz')['xpow']
    print (su, rest_file.shape)

    for win in range(nwin):
        for freq in range(1, nfreq):
            odor_data = np.swapaxes(odor_file[freq,:,win,:], 0,1)
            rest_data = np.swapaxes(rest_file[freq,:,0,:], 0, 1)
            print (odor_data.shape, rest_data.shape)
            elec = np.load(path.join(elecfiles, su+'_E1E2_concat_all_bipo_new.npz'))['channel']
            elec_label = np.load(path.join(elecfiles, su+'_E1E2_concat_all_bipo_new.npz'))['label']
            elec_label = np.array(elec_label, dtype='<U9')
            print ('Channels : ', elec.shape, elec.dtype,
                      'Labels : ', elec_label.shape, elec_label.dtype)

            #create a data matrix, concatenate along the trial dimension
            odor_rest = np.concatenate((odor_data, rest_data), axis=0)
            print ('Size of the concatenated data: ', odor_rest.shape,
                      'Number of features : ', odor_rest.shape[1])

            #create label vector (0 for rest and 1 for odor)
            label = [1]*odor_data.shape[0] + [0]*rest_data.shape[0]
            print ('Size of label for classif: ', len(label))

            # Define a cross validation:
            cv = defCv(label, n_folds=10, cvtype='skfold', rep=10)
            print (cv)

            # Define classifier technique
            clf = defClf(label, clf='lda') #,n_tree=200, random_state=100)
            print (clf)

            #Classify rest and odor
            cl = classify(label, clf=clf, cvtype=cv)

            # Evaluate the classifier on data:
            da,pvalue,daperm = cl.fit(odor_rest, grp=elec, n_perm=200,method='label_rnd')
            da_mean = np.mean(da,axis=0)
            print ('size decoding accuracy : ', da.shape)

            #Save information
            np.save(path2save+su+'_da_'+freq_bands[freq]+'_Window_'+str(win)+'_'+classifier, da)
            np.save(path2save+su+'_pvalue_'+freq_bands[freq]+'_Window_'+str(win)+'_'+classifier, pvalue)
            np.save(path2save+su+'_da_perm_'+freq_bands[freq]+'_Window_'+str(win)+'_'+classifier,daperm)

            #Generate the figure and save the plot
            title = 'Classif_Encoding_'+su+'_'+freq_bands[freq]+'_Window_'+str(win)+'_p<0.01'

            fig1 = plt.figure(1, figsize=(20,10))
            fig1.suptitle(title, fontsize="x-large")
            cl.daplot(da_mean, daperm=daperm, chance_method='perm', rmax=['top', 'right'],
            dpax=['bottom', 'left'], cmap='magma', ylim=[40,100], chance_unique=True, chance_level = 0.01,
            chance_color='darkgreen',)

            fname = path.join(path2save, su+'_DA_'+classifier+'_'+freq_bands[freq]+'_Window_'+str(win)+'_p.01.png')
            fig1.savefig(fname, dpi=300, bbox_inches='tight')
            print ('saving --»' ,fname)
            cl.info.to_excel(path2save+su+'_DA_'+classifier+'_'+freq_bands[freq]+'_Window_'+str(win)+'_p.01.xlsx')
            plt.clf()
            plt.close()
        del da_mean, da, daperm, pvalue, elec

-> Olfacto loaded
CHAF (6, 103, 7, 40)
CHAF (6, 103, 1, 40)
(40, 103) (40, 103)
Channels :  (103,) <U9 Labels :  (103,) <U9
Size of the concatenated data:  (80, 103) Number of features :  103
Size of label for classif:  80
10-times, 10 Stratified k-folds
LinearDiscriminantAnalysis(n_components=None, priors=array([ 0.5,  0.5]),
              shrinkage=None, solver='svd', store_covariance=False,
              tol=0.0001)
size decoding accuracy :  (10, 103)
saving --» /media/karim/Datas4To/Analyses_Intra_EM_Odor/Olfacto/classified/1_Classif_Windows_Encoding_th40_art400_30_250_by_label_500ms_windows/CHAF_DA_lda_theta_Window_0_p.01.png
(40, 103) (40, 103)
Channels :  (103,) <U9 Labels :  (103,) <U9
Size of the concatenated data:  (80, 103) Number of features :  103
Size of label for classif:  80
10-times, 10 Stratified k-folds
LinearDiscriminantAnalysis(n_components=None, priors=array([ 0.5,  0.5]),
              shrinkage=None, solver='svd', store_covariance=False,
              tol=0.0001